In [1]:
# Dependencies
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import yfinance as yf

Pull in Stock Data

In [54]:
# TODO: enter your ETF ticker, dates, and price interval
# set parameters for data pull
ticker = 'IWV'

# access data and display
df_etf = yf.download(ticker, interval='1d', period='max')

# Calculate daily returns [(new / old) - 1]
df_etf['ret'] = ((df_etf['Adj Close'] / df_etf['Adj Close'].shift()) - 1) * 100
df_etf = df_etf.reindex(columns=['Adj Close', 'ret'])

[*********************100%%**********************]  1 of 1 completed


Pull in Risk Free, FOMC Dates, Federal Funds Target Rate

In [56]:
# Pull in Fama/French 3 Factors [Daily] csv file
df_rf = pd.read_csv('RF.csv', skiprows=3, index_col=0, skipfooter=1)
df_rf.index = pd.to_datetime(df_rf.index, format='%Y%m%d', errors='coerce')
df_rf['RF'] /= 100

# Merge risk-free rate and etf data 
df_all = pd.merge(df_rf, df_etf, left_index=True, right_index=True, how='inner').dropna()
df_all.drop(['Mkt-RF', 'SMB', 'HML'], axis=1, inplace=True)

# Calculate excess returns
df_all['xs_ret'] = df_all['ret'] - df_all['RF']

# Pull in provided fomc data
df_fomc = pd.read_csv('FOMC_DATES.csv', index_col='Date')

# Reformat dates
df_fomc.index = pd.to_datetime(df_fomc.index, format='%Y%m%d', errors='coerce')

# Merge fomc dates
df_all = pd.merge(df_all, df_fomc, left_index=True, right_index=True, how='left')
df_all['fomc'] = df_all['fomc'].fillna(0)

# Reset index and make date column
df_all.index.name = 'Date'

# Pull Federal Funds Rate
df_ir = pd.read_csv('DFEDTAR.csv')
df_ir['Date'] = pd.to_datetime(df_ir['Date'])
df_ir.set_index('Date', inplace=True)

# Manipulate Federal Funds Rate
df_ir = df_ir.rename(columns={'interest rate': 'rate_after'})
df_ir['rate_before'] = df_ir['rate_after'].shift(1)
conditions = [
    (df_ir['rate_after'] > df_ir['rate_before']),
    (df_ir['rate_after'] < df_ir['rate_before']),
    (df_ir['rate_after'] == df_ir['rate_before'])
]
actions = ['raise', 'lower', 'nothing']
df_ir['action'] = np.select(conditions, actions, default='unknown')

# df_ir.index = pd.to_datetime(df_ir.index, format='%Y-%m-%d')
df_all = pd.merge(df_all, df_ir, left_index=True, right_index=True, how='left')
df_all


C:\Users\andre\AppData\Local\Temp\ipykernel_10260\209505247.py:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skipfooter; you can avoid this warning by specifying engine='python'.
  df_rf = pd.read_csv('RF.csv', skiprows=3, index_col=0, skipfooter=1)


,RF,Adj Close,ret,xs_ret,fomc,rate_after,rate_before,action
Date,,,,,,,,
2000-05-30,0.00023,52.222084,2.864264,2.864034,0.0,6.50,6.50,nothing
2000-05-31,0.00023,52.780560,1.069424,1.069194,0.0,6.50,6.50,nothing
2000-06-01,0.00018,52.928070,0.279479,0.279299,0.0,6.50,6.50,nothing
2000-06-02,0.00018,54.614025,3.185370,3.185190,0.0,6.50,6.50,nothing
2000-06-05,0.00018,54.329510,-0.520957,-0.521137,0.0,6.50,6.50,nothing
...,...,...,...,...,...,...,...,...
2023-07-25,0.00022,260.756927,0.233537,0.233317,0.0,5.25,5.25,nothing
2023-07-26,0.00022,260.995941,0.091661,0.091441,1.0,5.25,5.25,nothing
2023-07-27,0.00022,259.073761,-0.736479,-0.736699,0.0,5.50,5.25,raise
